In [1]:
import cv2
import os
import psycopg2
import numpy as np

In [3]:
from tensorflow.keras.applications import VGG16

def extract_and_store_features(image_paths, db_connection):
    # Load the VGG16 model
    model = VGG16(weights='imagenet', include_top=False)

    # Create a cursor object
    cur = db_connection.cursor()

    # Iterate over the image paths
    for image_path in image_paths:
        # Load the image
        img = cv2.imread(image_path)

        # Preprocess the image
        img = cv2.resize(img, (224, 224))
        img = img / 255.0

        # **Extract features from the image**
        features = model.predict(img[np.newaxis, :])  # Extract features (7, 7, 512)
        features = features.flatten().astype(float)  # Flatten to (25088,)

        # Store the features in the database
        cur.execute("INSERT INTO images_t (image_path, features) VALUES (%s, %s)", (image_path, features.tolist()))
        db_connection.commit()

    # Close the cursor object
    cur.close()

# Example usage:
db_connection = psycopg2.connect(
    host="localhost",
    database="Images",
    user="postgres",
    password="lass93"
)


"""
1. Install PostgresQL and set it up to store images
2. Write a function that take image paths, converted features and store them into a dB
3. Write a function that matches a random image and compare with the dB
"""

'\n1. Install PostgresQL and set it up to store images\n2. Write a function that take image paths, converted features and store them into a dB\n3. Write a function that matches a random image and compare with the dB\n'

In [13]:
image_paths = []

# Specify the path to the folder
folder = 'C:/Users/freddie/Desktop/dogs'

# List all files and folders in the directory
files_and_folders = os.listdir(folder)

# Print the full path of each item in the folder
for item in files_and_folders:
    full_path = os.path.join(folder, item)
    image_paths.append(full_path)

In [19]:
extract_and_store_features(image_paths, db_connection)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 625ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 446ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 444ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 444ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 456ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 446ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 445ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 465ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 465ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 447ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [5]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

def match_image(image_path, db_connection):
    """
    Extract features from an image and query PostgreSQL to find the most similar image.
    
    Args:
        image_path (str): Path to the image to be matched.
        db_connection (psycopg2 connection): Connection to the PostgreSQL database.
        
    Returns:
        dict: Information about the most similar image.
    """
    # **Load the VGG16 model**
    model = VGG16(weights='imagenet', include_top=False)

    # **Preprocess the input image**
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # Resize to VGG16 input size
    img = img / 255.0  # Normalize to [0, 1]

    # **Extract features from the image**
    feature = model.predict(img[np.newaxis, :]).flatten().astype(float)
        
    # **Query PostgreSQL for the closest image**
    cur = db_connection.cursor()

    query = """
    SELECT image_path, features AS similarity 
    FROM images_t 
    WHERE %s = features;
    """
    cur.execute(query, (feature,))  # Pass the image features to PostgreSQL
    result = cur.fetchone()

    cur.close()
    return {'image_path': result[0], 'similarity': result[1]}

# Usage
db_connection = psycopg2.connect(
    host="localhost",
    database="Images",
    user="postgres",
    password="lass93"
)



In [7]:
image_path = "C:/Users/freddie/Desktop/dog_breeds/images/n02085620-Chihuahua/n02085620_7.jpg"

matched_image = match_image(image_path, db_connection)
print(f"Best match: {matched_image}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 709ms/step


ProgrammingError: can't adapt type 'numpy.ndarray'

In [3]:

git branch -M main
git remote add origin https://github.com/atenidegbeolanrewaju/image_db.git
git push -u origin main

SyntaxError: invalid syntax (2686549568.py, line 1)